# Stock Exchange Example

## Install Requirements

In [1]:
!pip install --quiet agent-exchange==0.0.2 pandas numpy

In [2]:
from random import shuffle, random, randint
from typing import Sequence, Tuple, Dict, Union

import numpy as np
import pandas as pd
from agent_exchange.agent import Agent
from agent_exchange.exchange import Exchange

from stock_exchange_v1 import *

## Make Stock Exchange Agents

In [26]:
# Make stock exchange agents -- this draws on a number of 
class StockAgentV1InternalState:
    def __init__(self, initial_num_shares: int, initial_capital: float):
        self.num_shares = [initial_num_shares]
        self.capital = [initial_capital]
        
        # Mappings from price to number of shares
        self.open_bids = {}
        self.open_asks = {}

    def on_timestep_passed(self, fill_ticket: Union[type(None), StockExchangeV1FillTicket]):
        if fill_ticket != None:
            print('at 13')
            new_num_shares, new_capital = self.internal_state.update_with_fill_ticket(fill_ticket)
        else:
#             print('at 16')
            new_num_shares, new_capital = self.get_num_shares(), self.get_capital()
            
        self.num_shares.append(new_num_shares)
        self.capital.append(new_capital)
        
        
        
    def update_with_fill_ticket(self, ticket: StockExchangeV1FillTicket):
        """Use the fill ticket to update our state variables.
        """
        
        # The state updates for after the update -- this should be modified in this function when capital or num_shares changes
        new_num_shares = self.get_num_shares()
        new_capital = self.get_capital()

        # Add new bids
        for price in ticket.open_bids:
            StockAgentV1InternalState.increment_or_create(self.open_bids, price, ticket.open_bids[price])

        # Add new asks
        for price in ticket.open_asks:
            StockAgentV1InternalState.increment_or_create(self.open_asks, price, ticket.open_asks[price])

        # Remove old bids that were filled in the past time step
        for price in ticket.closed_bids:
            shares_bought = ticket.closed_bids[price]
            new_num_shares += shares_bought
            new_capital -= price * shares_bought
            StockAgentV1InternalState.decrement_and_try_delete(self.open_bids, price, shares_bought)
            

        # Remove old asks that were filled in the past time step
        for price in ticket.closed_asks:
            shares_sold = ticket.closed_asks[price]
            new_num_shares -= shares_sold
            new_capital += price * shares_sold
            StockAgentV1InternalState.decrement_and_try_delete(self.open_asks, price, shares_sold)
            
        return new_num_shares, new_capital

    def get_num_shares(self):
        return self.num_shares[-1]
    
    def get_capital(self):
        return self.capital[-1]

    def __repr__(self):
        return dict_repr(self)

    def __str__(self):
        return dict_str(self)

    def increment_or_create(D, key, value):
        """If the key-value pair does not exist yet,
        then add a new key-value pair with `value`
        as the value. Otherwise, increment the
        key's value with `value`.
        """
        if key not in D:
            D[key] = 0
        D[key] += value

    def decrement_and_try_delete(D, key, value):
        """Decrement a value in a dictionary,
        and if the new value is 0, then delete
        the k-v pair from the dictionary.
        """
        D[key] -= value
        if D[key] == 0:
            del D[key]

class StockAgentV1(Agent):
    """A base stock trading agent; this agent itself will perform no-ops each iteration.
    """
    def __init__(self, initial_num_shares, initial_capital):
        super().__init__()
        self.internal_state = StockAgentV1InternalState(initial_num_shares, initial_capital)

    def action_results_update(
        self, 
        new_order_book: StockExchangeV1OrderBook, 
        reward, 
        done: bool, 
        fill_ticket: Union[type(None), StockExchangeV1FillTicket]):
        
        self.internal_state.on_timestep_passed(fill_ticket)
        
    def __repr__(self):
        return dict_repr(self)

    def __str__(self):
        return dict_str(self)

    
class StockAgentV1NaiveMaker(StockAgentV1):
    """Naive agent that acts as a market maker.
    If there is a spread, this agent will place
    an order on the buy side 1/2 of the time, on
    the sell side 1/2 of the time.

    Note: in V1, there is no way to place multiple
    orders within the same time step.
    """
    def __init__(self, initial_num_shares=1000, initial_capital=100000):
        super().__init__(initial_num_shares, initial_capital)

    def get_action(self, order_book: StockExchangeV1OrderBook):
        if random() < .5: # buy just over current bid
            buy_price = order_book.get_bid() + 0.01 # penny-up on the market-clearing buy price
            
            # Randomly buy as little as 0 or as much as we can
            buy_amount = randint(0, 1000) # limit buy

            order = StockExchangeV1Action(StockExchangeV1OrderTypes.LIMIT, buy_amount, buy_price)

            return order
        else: # sell just under current ask
            sell_price = order_book.get_ask() - 0.01

            # Randomly sell as little as 0 and as much as 10
            sell_amount = randint(0, 1000)

            order = StockExchangeV1Action(StockExchangeV1OrderTypes.LIMIT, -sell_amount, sell_price)

            return order


class StockAgentV1NaiveTaker(StockAgentV1):
    """Naive agent that acts as a liquidity
    taker. This agent speculates by placing
    market orders of buy 1/2 of the time and
    sell 1/2 of the time. Here we ignore
    constraints on short selling, allowing
    agents to shorts sell without limit.

    Also, if the taker's order exhausts the
    order book, then only the portion of their
    order in the order book gets filled.
    """
    def __init__(self, initial_num_shares=1000, initial_capital=100000):
        super().__init__(initial_num_shares, initial_capital)

    def get_action(self, order_book: StockExchangeV1OrderBook):
        if random() < .5: # buy
            expected_buy_price = order_book.get_ask()
            
            # Randomly decide on how much to buy
            max_buy_amount = self.internal_state.get_capital() // expected_buy_price
            num_shares_to_buy = randint(0, max_buy_amount)

            return StockExchangeV1Action(StockExchangeV1OrderTypes.MARKET, num_shares_to_buy)

        else:
            expected_sell_price = order_book.get_bid()

            # Randomly decide on how much to sell
            max_sell_amount = self.internal_state.get_num_shares()
            num_shares_to_sell = randint(0, max_sell_amount)

            return StockExchangeV1Action(StockExchangeV1OrderTypes.MARKET, -num_shares_to_sell)

## Run simulations

In [28]:
# Run a simple 
NMAKER, NTAKER, NSTEPS = 10, 1, 10
agents = [StockAgentV1NaiveMaker() for _ in range(NMAKER)]
agents += [StockAgentV1NaiveTaker() for _ in range(NTAKER)]
exchange = StockExchangeV1(agents)

exchange.simulate_steps(NSTEPS)
# print(exchange.order_book)

In [29]:
agents

[{'internal_state': {'num_shares': [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], 'capital': [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000], 'open_bids': {}, 'open_asks': {}}},
 {'internal_state': {'num_shares': [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], 'capital': [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000], 'open_bids': {}, 'open_asks': {}}},
 {'internal_state': {'num_shares': [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], 'capital': [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000], 'open_bids': {}, 'open_asks': {}}},
 {'internal_state': {'num_shares': [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], 'capital': [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000], 'open_bids': {}, 'open_asks': {}}},
 {'internal_state': {'num_shares': [1000

In [25]:
agents[-1].internal_state.get_num_shares()

1000

In [43]:
help(exchange._StockExchangeV1__get_info)

Help on method __get_info in module stock_exchange_v1:

__get_info(agent_index) method of stock_exchange_v1.StockExchangeV1 instance
    Relay information about how many shares 
    of the user's previous orders filled.



In [34]:
class A:
    def __init__(self):
        pass
    def __hello(self):
        print("A says hello!")
class B(A):
    def __hello(self):
        print("B says hello!")

    def hiya(self):
        self.__hello()

In [36]:
B().hiya()

B says hello!
